# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**\
**Auxiliar: Rodrigo Lara M**

**17/05/2020 - Tarea 1**


**Integrantes del grupo**: Pablo Araya Z., Fabian Badilla M.

## 1 Introducción
La siguiente evaluación corresponde a la primera tarea del curso de laboratorio de ciencia de datos. A modo de contexto, se trabajará con un conjunto de datos, en el cual se busca una estimación del precio por metro cuadrado para viviendas en la ciudad de Bogotá, se busca además de una estimación de incertidumbre en predicción.\
Para ello, se proporcionan datos recolectados por _web-scrapping_ y una serie de estadísticas que caracterizan determinadas zonas geográficas de interés, llamadas Unidades de Planeación Zonal (UPZ).

Las condiciones de entrega requeridas son:

* La extensión máxima de el informe es de 6 planas a las que puede añadir 2 para demostraciones.
* Debe adjuntar un repositorio `git` donde se incluya todo su código.
* A lo menos 1 `commit` por cada pregunta de la tarea
* Por lo menos 1 `merge` a través de su trabajo.
* Incluya un documento `jupyter notebook` llamado `tarea1.ipynb` en el cual se exponga todo el procedimiento realizado.
* Por último es necesario también entregar un archivo _pickle_ denominado `modelo.pk` que contenga el último modelo de regresión entrenado.

Tenga en mente que su informe será revisado por un equipo técnico que debe entender a cabalidad su metodología, ser capaz de replicarlo y evaluarlo a partir de la lectura de este.

## Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os, sys

## P1. Carga y limpieza de datos
En la presente sección se realizan los pasos de carga y limpieza de datos que permitirán realizar las secciones
posteriores con un `DataFrame` consolidado que presente los tipos de datos adecuados para la información contenida
en sus columnas.\
Incluya en el reporte todas las decisiones que llevó en esta seccion, ademas de reportar y discutir acerca de
los aspectos específicos señalados en cada pregunta.

1. Los datos recolectados en la carpeta `data/raw` están divididos en carpetas con el formato `wNN` donde `NN` corresponde a la semana del año en la que estos fueron consultados. Cargue los datos en un solo `DataFrame` y elimine las filas duplicadas. Además genere una variable categórica en la que se indique si la observación correspondiente proviene de un archivo que en su nombre contiene `'furnished'`\
(ejemplo: metrocuadrado_furnished_wNN.csv).\
**Hint:** para lo último puede ser útil estudiar los argumentos del
método `pd.merge`. Reporte si existen observaciones de archivos con texto `'furnished'` que no estén contenidos en archivos con texto `'all'`.

In [4]:
df_s = []
for i in range(13,18):
    w = "w"+str(i)
    direccion = 'data/raw/'+w+'/metrocuadrado_all_'+w+'.csv'
    df = pd.read_csv(direccion)
    df_s.append(df)
df_all = pd.concat(df_s,axis=0,ignore_index = True)

'''Creamos la variable categorica'''
df_all['Archivo'] = 'all'

''' Eliminamos los datos duplicados'''
df_all.drop_duplicates(inplace=True,ignore_index=True)

df_s = []
for i in range(13,18):
    w = "w"+str(i)
    direccion = 'data/raw/'+w+'/metrocuadrado_furnished_'+w+'.csv'
    df = pd.read_csv(direccion)
    df_s.append(df)
df_fur = pd.concat(df_s,axis=0,ignore_index = True)
df_fur['Archivo'] = 'furnished'
df_fur.drop_duplicates(inplace=True,ignore_index=True)

df = pd.concat([df_all,df_fur],axis=0,ignore_index = True)
df.head()

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


2. Limpie las columnas:
    1. Precio y de área del inmueble, número de habitaciones y número de baños.
    2. De `'property type|rent type|location'`. Debe obtener 3 columnas en las que se detalle el tipo de inmueble (casa o apartamento), el tipo de la oferta (arriendo o arriendo y venta) además de el barrio en el cual se ubica el inmueble (texto en mayúsculas para la mayoría de los casos) Llame a esta última columna `'location'`.\
    __Hint:__ Para ello pueden ser útiles los métodos `str.split` y `str.strip` de la clase `pd.Series`.

In [5]:
df = df.dropna(axis = 0,subset=['price','n_rooms','n_bath','surface'])
df.reset_index(drop=True,inplace=True)

df['price'] = [x.replace('$','') for x in df['price']]
df['price'] = [x.replace('.','') for x in df['price']]
df['price'] = df['price'].astype('float64')
df['surface'] = [x.replace(' m2','') for x in df['surface']]
df['surface'] = df['surface'].astype('float64')

df.dtypes

property_type|rent_type|location     object
price                               float64
n_rooms                              object
n_bath                               object
surface                             float64
details                              object
url                                  object
metrocuadrado_index                 float64
Archivo                              object
dtype: object

In [6]:
columna = 'property_type|rent_type|location'
first_col = df[columna]
first_col = pd.Series([x.replace(' ',', ', 1) for x in first_col])
first_col = first_col.str.split(pat=',',expand =True)
first_col.columns = columna.split('|')
first_col['rent_type'] =  [x.replace('en ','') for x in first_col['rent_type']]
df = pd.concat([first_col,df.iloc[:,1:]],axis=1)

3. Adjunte las siguientes columnas:
    1. Genere una variable que represente el precio por metro cuadrado.
    2. Obtenga el número de garajes procesando la columna `'url'` .

In [7]:
df['Precio_m2'] = df['price']/df['surface']

In [15]:
lista = []
for x in df['url']:
    if '-garajes' not in x:
        lista.append('0')
    else:
        lista.append(x[x.find('banos-')+6:x.find('-garajes')])
df['Numero_Garajes'] = lista

4. Clasifique las observaciones por tipo de producto de acuerdo a los criterios de la Tabla
    1. Hint: puede ser útil el método `query` de `pd.DataFrame`

|Tipo de Producto| Tipo de Inmueble| área min | área max |
|----------------|-----------------|----------|----------|
|        1       |       casa      | 80       |    <120  |
|        2       |       casa      | 120      |    <180  |
|        3       |       casa      | 180      |    <240  |
|        4       |       casa      | 240      |    <360  |
|        5       |       casa      | 360      |    460   |
|        6       |   apartamento   | 40       |    <60   |
|        7       |   apartamento   | 60       |    <80   |
|        8       |   apartamento   | 80       |    120   |

In [19]:
condiciones = ['80<=surface<120 and property_type=="Casa"','120<=surface<180 and property_type=="Casa"',
              '180<=surface<240 and property_type=="Casa"','240<=surface<360 and property_type=="Casa"',
              '360<=surface<=460 and property_type=="Casa"','40<=surface<60 and property_type=="Apartamento"',
              '60<=surface<80 and property_type=="Apartamento"','80<=surface<=120 and property_type=="Apartamento"']
df['tipo_producto'] = np.zeros(len(df))
for i in range(1,9):
    ind = df.query(condiciones[i-1]).index
    df.loc[ind,'tipo_producto']=i

5. A partir de la columna `'barrio'`, haga una fusión con el archivo `data/assignacion upz/barrio-upz-asignacion.csv` para obtener así el código de la UPZ de cada inmueble. Reporte el numero de observaciones y de barrios a los que no se les puede adjuntar un código UPZ a partir de este archivo. Tenga en cuenta que aproximadamente 90% de los datos tiene información sobre UPZ.

In [10]:
barrio_upz =pd.read_csv('data/asignacion_upz/barrio-upz.csv')
barrio_upz

,UPlCodigo,UPlTipo,UPlNombre,UPlArea,pro_location
0,UPZ100,1,GALERIAS,2.375681e+06,palermo
1,UPZ100,1,GALERIAS,2.375681e+06,belalcazar
2,UPZ100,1,GALERIAS,2.375681e+06,sears
3,UPZ100,1,GALERIAS,2.375681e+06,galerias
4,UPZ100,1,GALERIAS,2.375681e+06,banco central
...,...,...,...,...,...
463,UPZ42,1,VENECIA,6.638420e+06,nuevo muzu
464,UPZ42,1,VENECIA,6.638420e+06,el condor
465,UPZ42,1,VENECIA,6.638420e+06,laguneta
466,UPZ52,1,LA FLORA,1.877160e+06,villa maria


6. En la carpeta `data/estadisticas_upz` encontraría todos los archivos que debe fusionar con su `DataFrame`, a través del código de UPZ, para así enriquecer su conjunto de datos con estadísticas de población, socioeconómicas y de calidad de vida a nivel de UPZ. Una vez realizada la fusión, adjunte una nueva columna con la densidad de población por UPZ.

## P2. EDA
En la siguiente pregunta utilice el `DataFrame` obtenido a partir de los procedimientos anteriores. La idea central del presente ejercicio es analizar la base de datos que fue construida realizando un análisis exploratorio de datos (EDA) riguroso que permite obtener información útil para la parte final de tarea. Para cada pregunta y considerando que la variable de respuesta es el precio por metro cuadrado, discuta sus resultados y reporte algunos gráficos interesantes en su informe.

1. Programe una función `estilo()` que aplica un estilo de gráficos por defecto diseñado por usted. Este estilo debe ser empleado en todos los gráficos que incluya en su informe.\
__Hint:__ Use métodos de la librería `seaborn` para ello.

2. Perfile las variables obtenidas en su `DataFrame`, agrúpelas y analícelas según su naturaleza. En función de su agrupación, grafíque las distribuciones dando un tratamiento adecuado a cada tipo de variable, discuta ciertos casos de interés.

3. Estudie la presencia de datos faltantes en la base de datos. Observe como se distribuyen estos y establezca un mecanismo de pérdida de información basándose en los patrones observables del conjunto de datos. Busque agrupaciones de columnas que muestren un comportamiento sistemático y plantee sus reflexiones.\
Respalde con visualizaciones y cuantifique estadísticamente los patrones observados.

4. Recategoríce la variable código de UPZ de forma que quede distribuida entre 3 a 5 grupos, evalúe la significancia estadística de esta nueva agrupación en comparación a la variable de respuesta. Comente sus resultados e interprételos.\
__Hint:__ Puede probar con técnicas de clustering (como k-means) sobre una agrupación de UPZ y validar estadísticamente si las nuevas categorías afectan la variable de respuesta.

5. Cuantifique estadísticamente las relaciones entre una selección de al menos 10 variables de interés y la columna de respuesta, examine también las relaciones entre las variables de su selección. Utilice las herramientas de análisis estadístico que considere pertinentes, comente brevemente sus hallazgos.

6. En base a las herramientas del curso realice un análisis que permita detectar observaciones anómalas en la base de datos, justifique sus resultados, evalúe como se distribuyen los valores anómalos respecto a las variables UPZ y tipo de producto.

7. En función del análisis realizado a lo largo de esta pregunta, proponga una selección de variables que permita estimar la variable respuesta por medio de un modelo de regresión, discuta.

## P3. Regresión lineal bayesiana
Una vez analizado el conjunto de datos, se procede a modelar las relaciones por medio de alguna herramienta matemática. En este contexto y con las herramientas computacionales entregadas por el curso, se desarrolla un modelo de regresión lineal bayesiana, tratando el problema de modelación desde el punto vista teórico hasta su implementación e interpretación de resultados.

#### Implementación
1. Implemente la clase `RegresionBayesianaEmpirica` que herede de `BaseEstimator` y de `RegressorMixin` del módulo `sklearn.base` en la cual se implementa la heurística enunciada en la sección anterior para aproximar los hiperparámetros óptimos $\alpha$ y $\beta$. Esta clase sólo debe usar objetos de la librería `NumPy` y debe incluir al menos los siguientes métodos:
    * `__init__(self, alpha 0, beta 0, tol=1e-5, maxiter=200)`: sus argumentos son auto explicativos.
    * `get_posteriori(self, X, y, alpha, beta)`: que reciba la matriz de observaciones (`X`), el vector de etiquetas (`y`) y los hiperparámetros $\alpha$ y $\beta$. Este debe retornar los objetos necesarios para interactuar con los demás métodos.
    * `fit(self, X, y)`: que reciba la matriz de observaciones (`X`), el vector de etiquetas (`y`) e implemente el esquema de aproximación mencionado. Este método debe guardar como atributos del objeto los parámetros óptimos obtenidos, además de reportar en pantalla indicadores del proceso iterativo (incluya al menos el número de iteraciones).
    * `predict(self, X , return std=False)`: que reciba una matriz de observaciones (`X_`). Debe retornar la tupla (`y_` , `y_std`) con el vector de medias y el de desviaciones estándar (cuando `return std=True`) asociadas a las observaciones en `X`. Para esto, observe que el proceso de predicción corresponde a asignar la media posterior predictiva del modelo a nuevos puntos.
    
   Note que todo desarrollo sólo necesita de un modelo lineal en los parámetros $w$. Es decir, es posible reemplazar $X$ por una transformación (posiblemente no lineal) $\Phi(X)$, manteniendo la misma estructura distrubucional tanto en predicción como en obtención de hiperparámetros.

En las siguientes preguntas se construye un modelo de regresion en el cual debe incluir todas las variables disponibles en el conjunto de datos, reemplazando la variable de UPZ por la recategorizacion antes propuesta.

2. Construya un fujo de transformaciones sobre el conjunto datos. Por medio de la clase `Pipeline` debería:
    * Utilizar `StandardScaler` y `OneHotEncoder` donde corresponda.
    * Utilizar el objeto `PolynomialFeatures` para generar características polinomiales, en este apartado, se recomienda utilizar características de grado 3 sólo en las variables numéricas y luego concatenar con las codificaciones categóricas.
    * Generar una composición de transformaciones por medio de `ColumnTransformer`.

3. Expanda el `Pipeline` anterior agregando el modelo representado en la clase `RegresionBayesianaEmpirica`. Genere una separación de entrenamiento y test por medio de `train_test_split` del módulo `model_selection` donde el 20% de los datos sea de test. Entrene su modelo utilizando el método .fit como parámetros dentro del flujo creado por medio de Pipeline, evalúe su modelo por medio de la raíz del promedio de errores cuadráticos (_Root Mean Square Error - RMSE_. en inglés) en el conjunto de test, incluya el estadístico $R^2$.\
__Hint__: pruebe con valores para `alpha_0` y `beta_0` entre $10^{-10}$ y $10^{-5}$. Compruebe que dichos dichos hiperparametros son adecuados, al obtener un $R^2$ cercano a $0.7$.

4. Utilice su selección de variables, transfórmelas adaptando el esquema de preprocesamiento anterior, separe en conjuntos de entrenamiento y test manteniendo el 20% de proporción y evalúe los resultados del modelo `RegresionBayesianaEmpirica` incluyendo el estadístico $R^2$. Discuta.

5. Finalmente, compare los resultados de su selección de variables con los obtenidos por un pipeline donde el estimador sea uno de la clase `BayesianRidge` del módulo `sklearn.linear_model`. Discuta la diferencia de este módulo con respecto al desarrollado por ustedes.